<a href="https://colab.research.google.com/github/aleeepassarelli/scientific-validation-hub/blob/main/notebooks/03_metadata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📦 Lab 03: Metadata Packaging com RO-Crate

**Framework:** Scientific Validation Hub
**Ferramenta:** [RO-Crate (Research Object Crate)](https://www.researchobject.org/ro-crate/)
**Objetivo:** Transformar arquivos soltos em um pacote científico padronizado (FAIR), garantindo que sua autoria e contexto sejam preservados por máquinas e humanos.

---

### ⚡ Quick Start (Modo Assistido)
1.  **Simule:** Rode as células para criar arquivos de pesquisa falsos e empacotá-los com metadados.
2.  **Visualize:** O código gerará um arquivo JSON-LD complexo (o "cérebro" do pacote).
3.  **Audite:** Abra o Assistente Gemini e cole:

> "Atue como Bibliotecário de Dados Científicos.
> Analise o JSON-LD gerado abaixo (ro-crate-metadata.json).
> 1. O pacote identifica corretamente quem é o autor ('author')?
> 2. Existe uma licença de uso definida?
> 3. Os arquivos 'dataset.csv' e 'script.py' estão devidamente registrados?
> Se tudo estiver correto, confirme que este pacote é FAIR (Findable, Accessible, Interoperable, Reusable)."

In [ ]:
# Instala a biblioteca oficial do RO-Crate
!pip install rocrate -q
print("✅ Biblioteca RO-Crate instalada. Pronto para empacotar.")

In [ ]:
import os
import pandas as pd

# 1. Preparando o Terreno
# Vamos criar uma pasta simulando um projeto que acabou de ser concluído
project_dir = "meu_experimento_final"
if not os.path.exists(project_dir):
    os.makedirs(project_dir)

# 2. Criando Artefatos (Os arquivos da pesquisa)
# Arquivo A: Um dataset CSV
df = pd.DataFrame({'id': [1, 2], 'resultado': [0.98, 0.99]})
df.to_csv(f"{project_dir}/resultados_finais.csv", index=False)

# Arquivo B: O script que gerou os dados
script_code = "print('Calculando a resposta para a vida, o universo e tudo mais...')"
with open(f"{project_dir}/analise.py", "w") as f:
    f.write(script_code)

print(f"📂 Projeto simulado criado em: {project_dir}/")
print("   - resultados_finais.csv")
print("   - analise.py")
print("\n⚠️ PROBLEMA: Se eu te mandar essa pasta zipada agora, você não sabe quem sou eu nem a licença.")

In [ ]:
from rocrate.rocrate import ROCrate
from rocrate.model.person import Person
from rocrate.model.contextentity import ContextEntity
import json

# 3. Iniciando o RO-Crate
crate = ROCrate()
crate.name = "Validação de Agentes SLE v1.1"
crate.description = "Dataset contendo métricas de validação de densidade semântica para agentes de IA."

# 4. Adicionando Identidade (Autoria)
# Isso garante que você seja citado se alguém usar seus dados
author = crate.add(Person(crate, properties={
    "name": "Hacker Semântico",
    "affiliation": "Scientific Validation Hub",
    "email": "researcher@hub.com"
}))
crate.creator = author

# 5. Adicionando Licença (Jurídico)
license_spdx = crate.add(ContextEntity(crate, "https://spdx.org/licenses/MIT", properties={
    "name": "MIT License"
}))
crate.license = license_spdx

# 6. Registrando os Arquivos (Provenance)
# Não basta o arquivo estar lá, precisamos descrever O QUE ele é.
file_dataset = crate.add_file(
    source=f"{project_dir}/resultados_finais.csv",
    properties={
        "name": "Resultados de Validação",
        "encodingFormat": "text/csv",
        "description": "Tabela contendo scores de densidade semântica."
    }
)

file_script = crate.add_file(
    source=f"{project_dir}/analise.py",
    properties={
        "name": "Script de Análise",
        "encodingFormat": "application/x-python",
        "programmingLanguage": "Python 3.9"
    }
)

# 7. Salvando o Pacote
output_folder = "crate_empacotado"
crate.write(output_folder)

print(f"📦 PACOTE GERADO COM SUCESSO EM: {output_folder}/")
print("   O arquivo vital 'ro-crate-metadata.json' foi criado.")

In [ ]:
# 8. Auditoria do JSON-LD
# Este é o arquivo que os robôs (Google Dataset Search, Zenodo) leem.
metadata_path = f"{output_folder}/ro-crate-metadata.json"

with open(metadata_path, "r") as f:
    metadata = json.load(f)

print("🔍 INSPEÇÃO DO METADATA (JSON-LD):")
print("="*60)
print(json.dumps(metadata, indent=2))
print("="*60)

print("\n✅ PRÓXIMO PASSO: Copie o JSON acima (ou peça para o Gemini ler) e valide a conformidade FAIR.")